# Problem 1

Customer who needs to make a delivery needs to know which courier company can does it 
fast (assuming that the shorter the distance, the quicker is the delivery). The application will analyse five 
(5) local courier companies which have their delivery hubs located in various locations in West Malaysia. 
The details of the courier companies and their delivery hubs are given

### 1. Get and mark locations of all hub locations given 

In [10]:
import pandas as pd
import gmplot
from IPython.display import IFrame
API_key = '****'

deliveryHub = pd.read_csv('AlgorithmDataCourier.csv')
deliveryHub['Latitude'][0]
#print(deliveryHub)
DHlat = deliveryHub['Latitude']
DHlong = deliveryHub['Longitude']

hubPoint =  gmplot.GoogleMapPlotter(DHlat[1],DHlong[1],10)
hubPoint.scatter(DHlat,DHlong,'red',size=50,marker=True)
hubPoint.marker(DHlat[0], DHlong[0], title="City-link Express, Port Klang")
hubPoint.marker(DHlat[1], DHlong[1], title="Pos Laju, Petaling Jaya")
hubPoint.marker(DHlat[2], DHlong[2], title="GDEX, Batu Caves")
hubPoint.marker(DHlat[3], DHlong[3], title="J&T, Kajang")
hubPoint.marker(DHlat[4], DHlong[4], title="DHL, Sungai Buloh")

hubPoint.apikey = API_key
hubPoint.draw('map.html')
IFrame(src='map.html', width=800, height=600)

### 2. Get the distances between origin and destination of the customers parcel details.

In [2]:
import googlemaps
gmaps = googlemaps.Client(key=API_key)

custDelivery_Data = pd.read_csv('AlgorithmDataCustomer.csv')

distanceOriDes = []
origin_Lat = custDelivery_Data['Latitude-Origin']
origin_Long = custDelivery_Data['Longitude-Origin']
dest_Lat = custDelivery_Data['Latitude-Destination']
dest_Long = custDelivery_Data['Longitude-Destination']

origin = (origin_Lat, origin_Long)
destination = (dest_Lat, dest_Long)

for i in range(len(custDelivery_Data['Latitude-Origin'])):
    origin = (origin_Lat[i], origin_Long[i])
    destination = (dest_Lat[i], dest_Long[i])
    distance = gmaps.distance_matrix(origin, destination, mode='driving')["rows"][0]["elements"][0]["distance"]["value"]
    distanceOriDes.append(distance/1000)
    
custDelivery_Data

,Customer,Origin,Latitude-Origin,Longitude-Origin,Destination,Latitude-Destination,Longitude-Destination
0,Customer 1,Rawang,3.361540,101.563182,Bukit Jelutong,3.100017,101.530715
1,Customer 2,Subang Jaya,3.049398,101.585466,Puncak Alam,3.227994,101.427304
2,Customer 3,Ampang,3.141856,101.761586,Cyberjaya,2.918870,101.652518


In [3]:
custDelivery_Data["Distance"] = distanceOriDes
custDelivery_Data

,Customer,Origin,Latitude-Origin,Longitude-Origin,Destination,Latitude-Destination,Longitude-Destination,Distance
0,Customer 1,Rawang,3.361540,101.563182,Bukit Jelutong,3.100017,101.530715,41.591
1,Customer 2,Subang Jaya,3.049398,101.585466,Puncak Alam,3.227994,101.427304,38.456
2,Customer 3,Ampang,3.141856,101.761586,Cyberjaya,2.918870,101.652518,35.958


### 3. Assuming that all deliveries must go through their delivery hub. For example, a delivery from Kuala Lumpur (origin) will go through Pos Laju Hub in Petaling Jaya to Putrajaya (destination). Suggest the least distance that the parcel has to travel for each customer using every courier company. 

#### a. Calculating total distance between Origin - Delivery Hub - Destination

In [4]:
fullData = pd.read_csv('AlgorithmDataFull.csv')

totalDistanceList = []
distance1List = []
distance2List = []

origin_Lat = fullData['Latitude-Origin']
origin_Long = fullData['Longitude-Origin']
dh_Lat = fullData['Latitude-DH']
dh_Long = fullData['Longitude-DH']
dest_Lat = fullData['Latitude-Destination']
dest_Long = fullData['Longitude-Destination']

origin = (origin_Lat, origin_Long)
del_hub = (dh_Lat, dh_Long)
destination = (dest_Lat, dest_Long)

for i in range(len(fullData['Latitude-Origin'])):
    origin = (origin_Lat[i], origin_Long[i])
    del_hub = (dh_Lat[i], dh_Long[i])
    destination = (dest_Lat[i], dest_Long[i])
    
    #calculate distance
    distance1 = gmaps.distance_matrix(origin, del_hub, mode='driving')["rows"][0]["elements"][0]["distance"]["value"]
    distance2 = gmaps.distance_matrix(del_hub, destination, mode='driving')["rows"][0]["elements"][0]["distance"]["value"]
    total_distance = distance1 + distance2
    
    distance1List.append(distance1/1000)
    distance2List.append(distance2/1000)
    totalDistanceList.append(total_distance/1000)
    
fullData["Distance1 (Km)"] = distance1List
fullData["Distance2 (Km)"] = distance2List
fullData["Total Distance (Km)"] = totalDistanceList
fullData.to_csv('AlgoCalculated_Distances.csv', sep=',', index=None, header= ['Cust','Origin','Latitude-Origin','Longitude-Origin','Delivery Hub','Latitude-DH','Longitude-DH','Destination','Latitude-Destination','Longitude-Destination','Distance1 (Km)','Distance2 (Km)','Total Distance (Km)'])
calc_distance = pd.read_csv('AlgoCalculated_Distances.csv')
calc_distance

,Cust,Origin,Latitude-Origin,Longitude-Origin,Delivery Hub,Latitude-DH,Longitude-DH,Destination,Latitude-Destination,Longitude-Destination,Distance1 (Km),Distance2 (Km),Total Distance (Km)
0,1,Rawang,3.361540,101.563182,Port Klang,3.031992,101.373441,Bukit Jelutong,3.100017,101.530715,70.218,28.814,99.032
1,1,Rawang,3.361540,101.563182,Petaling Jaya,3.112924,101.639826,Bukit Jelutong,3.100017,101.530715,42.783,20.457,63.240
2,1,Rawang,3.361540,101.563182,Batu Caves,3.265155,101.680248,Bukit Jelutong,3.100017,101.530715,24.754,36.345,61.099
3,1,Rawang,3.361540,101.563182,Kajang,2.944121,101.790152,Bukit Jelutong,3.100017,101.530715,75.744,49.207,124.951
4,1,Rawang,3.361540,101.563182,Sungai Buloh,3.212723,101.574673,Bukit Jelutong,3.100017,101.530715,26.780,27.174,53.954
5,2,Subang Jaya,3.049398,101.585466,Port Klang,3.031992,101.373441,Puncak Alam,3.227994,101.427304,39.679,30.570,70.249
6,2,Subang Jaya,3.049398,101.585466,Petaling Jaya,3.112924,101.639826,Puncak Alam,3.227994,101.427304,14.021,41.995,56.016
7,2,Subang Jaya,3.049398,101.585466,Batu Caves,3.265155,101.680248,Puncak Alam,3.227994,101.427304,35.918,41.104,77.022
8,2,Subang Jaya,3.049398,101.585466,Kajang,2.944121,101.790152,Puncak Alam,3.227994,101.427304,35.215,70.745,105.960
9,2,Subang Jaya,3.049398,101.585466,Sungai Buloh,3.212723,101.574673,Puncak Alam,3.227994,101.427304,24.145,23.378,47.523


#### b. Sort the distance to find the shortest path

In [5]:
# shell sort

def shellSort(array, n, arr):

    # Rearrange elements at each n/2, n/4, n/8, ... intervals
    interval = n // 2
    while interval > 0:
        for i in range(interval, n):
            temp = array[i]
            temp1 = arr[i]
            j = i
            while j >= interval and array[j - interval] > temp:
                array[j] = array[j - interval]
                arr[j] = arr[j - interval]
                j -= interval

            array[j] = temp
            arr[j] = temp1
        interval //= 2
        
#initialize
customers = calc_distance['Cust']   
total_distances = calc_distance['Total Distance (Km)']
del_hub = calc_distance['Delivery Hub']
size = len(total_distances)

#initialize sorted hub
customer1 = []
hub1 = []
customer2 = []
hub2 = []
customer3 = []
hub3 = []

for i in range(size):
    if customers[i] == 1:
        customer1.append(total_distances[i])
        hub1.append(del_hub[i])
    elif customers[i] == 2:
        customer2.append(total_distances[i])
        hub2.append(del_hub[i])
    elif customers[i] == 3:
        customer3.append(total_distances[i])
        hub3.append(del_hub[i])

#sort
shellSort(customer1, len(customer1), hub1)
shellSort(customer2, len(customer2), hub2)
shellSort(customer3, len(customer3), hub3)

#display
couriers1 = []
couriers2 = []
couriers3 = []

def deliveryHub(hub, couriers):
    for delivery_hub in hub:
        if delivery_hub == 'Port Klang':
            couriers.append('City-link Express')
        elif delivery_hub == 'Petaling Jaya ':
            couriers.append('Pos Laju')
        elif delivery_hub == 'Batu Caves':
            couriers.append('GDEX')
        elif delivery_hub == 'Kajang':
            couriers.append('J&T')
        elif delivery_hub == 'Sungai Buloh':
            couriers.append('DHL')

deliveryHub(hub1, couriers1)
deliveryHub(hub2, couriers2)
deliveryHub(hub3, couriers3)

print('Customer 1: Distance and Delivery Hub')
print(customer1)
print(hub1)
print(couriers1)
print("The Courier Company with the shortest distance is", couriers1[0], "(", customer1[0], "km )")

print('\nCustomer 2: Distance and Delivery Hub')
print(customer2)
print(hub2)
print(couriers2)
print("The Courier Company with the shortest distance is", couriers2[0], "(", customer2[0], "km )")

print('\nCustomer 3: Distance and Delivery Hub')
print(customer3)
print(hub3)
print(couriers3)
print("The Courier Company with the shortest distance is", couriers3[0], "(", customer3[0], "km )")

Customer 1: Distance and Delivery Hub
[53.954, 61.099, 63.24, 99.032, 124.951]
['Sungai Buloh', 'Batu Caves', 'Petaling Jaya ', 'Port Klang', 'Kajang']
['DHL', 'GDEX', 'Pos Laju', 'City-link Express', 'J&T']
The Courier Company with the shortest distance is DHL ( 53.954 km )

Customer 2: Distance and Delivery Hub
[47.523, 56.016, 70.249, 77.022, 105.96]
['Sungai Buloh', 'Petaling Jaya ', 'Port Klang', 'Batu Caves', 'Kajang']
['DHL', 'Pos Laju', 'City-link Express', 'GDEX', 'J&T']
The Courier Company with the shortest distance is DHL ( 47.523 km )

Customer 3: Distance and Delivery Hub
[52.392, 53.555, 72.432, 83.517, 111.776]
['Kajang', 'Petaling Jaya ', 'Batu Caves', 'Sungai Buloh', 'Port Klang']
['J&T', 'Pos Laju', 'GDEX', 'DHL', 'City-link Express']
The Courier Company with the shortest distance is J&T ( 52.392 km )


#### Store shortest path

In [6]:
pathLat1 = []
pathLong1 = []
pathLat2 = []
pathLong2 = []
pathLat3 = []
pathLong3 = []
delHub = fullData['Delivery Hub']


def findCoordinate(courier, pathLat, pathLong):
    for i in range(len(courier)):
        if courier[0] == 'DHL':
            for j in range(len(delHub)):
                if delHub[j] == 'Sungai Buloh':
                    pathLat.append(dh_Lat[j])
                    pathLong.append(dh_Long[j])
                    break
            break
        elif courier[0] == 'J&T':
            for j in range(len(delHub)):
                if delHub[j] == 'Kajang':
                    pathLat.append(dh_Lat[j])
                    pathLong.append(dh_Long[j])
                    break
            break
        elif courier[0] == 'Pos Laju':
            for j in range(len(delHub)):
                if delHub[j] == 'Petaling Jaya ':
                    pathLat.append(dh_Lat[j])
                    pathLong.append(dh_Long[j])
                    break
            break
        elif courier[0] == 'GDEX':
            for j in range(len(delHub)):
                if delHub[j] == 'Batu Caves':
                    pathLat.append(dh_Lat[j])
                    pathLong.append(dh_Long[j])
                    break
            break
        elif courier[0] == 'City-link Express':
            for j in range(len(delHub)):
                if delHub[j] == 'Port Klang':
                    pathLat.append(dh_Lat[j])
                    pathLong.append(dh_Long[j])
                    break
            break
    
    

pathLat1.append(origin_Lat[0])
pathLong1.append(origin_Long[0])
pathLat2.append(origin_Lat[5])
pathLong2.append(origin_Long[5])
pathLat3.append(origin_Lat[10])
pathLong3.append(origin_Long[10])    

findCoordinate(couriers1, pathLat1, pathLong1)
findCoordinate(couriers2, pathLat2, pathLong2)
findCoordinate(couriers3, pathLat3, pathLong3)

pathLat1.append(dest_Lat[0])
pathLong1.append(dest_Long[0])
pathLat2.append(dest_Lat[5])
pathLong2.append(dest_Long[5])
pathLat3.append(dest_Lat[10])
pathLong3.append(dest_Long[10])


### 4. Plot line between the shortest destinations before and after 

In [8]:
location = gmplot.GoogleMapPlotter(DHlat[1],DHlong[1],10)
location.scatter(DHlat, DHlong, 'red', size=50, marker=True)
location.scatter(origin_Lat, origin_Long, 'blue', size=50, marker=True)
location.scatter(dest_Lat, dest_Long, 'black', size=50, marker=True)

location.plot(pathLat1,pathLong1,'blue',edge_widht=3.5)
location.plot(pathLat2,pathLong2,'green',edge_widht=3.5)
location.plot(pathLat3,pathLong3,'black',edge_widht=3.5)

#marker for customer origin
location.marker(pathLat1[0], pathLong1[0], title="Customer 1 Origin")
location.marker(pathLat2[0], pathLong2[0], title="Customer 2 Origin")
location.marker(pathLat3[0], pathLong3[0], title="Customer 3 Origin")

#marker for ustomer destination
location.marker(pathLat1[2], pathLong1[2], title="Customer 1 Destination")
location.marker(pathLat1[2], pathLong1[2], title="Customer 2 Destination")
location.marker(pathLat1[2], pathLong1[2], title="Customer 3 Destination")

#marker for hub
location.marker(DHlat[0], DHlong[0], title="City-link Express, Port Klang")
location.marker(DHlat[1], DHlong[1], title="Pos Laju, Petaling Jaya")
location.marker(DHlat[2], DHlong[2], title="GDEX, Batu Caves")
location.marker(DHlat[3], DHlong[3], title="J&T, Kajang")
location.marker(DHlat[4], DHlong[4], title="DHL, Sungai Buloh")

location.apikey = API_key
location.draw('map1.html')
IFrame(src='map1.html', width=800, height=600)
